In [13]:
#function definitions

cel_dates <- function(cel_paths) {
    #IN: vector with paths to CEL files
    #OUT: vector of CEL scan dates
    scan_dates <- c()
    for (i in seq_along(cel_paths)) {
        datheader <- readCelHeader(cel_paths[i])$datheader
        scan_date <- gsub(".*([0-9]{2}/[0-9]{2}/[0-9]{2}).*", "\\1", datheader)
        scan_dates[i] <- scan_date       
    }
    return (scan_dates)
}


inputs <- function(instructions="", two=F, box1="AL group name", box2="CR group name"){
   #IN:
   #OUT:

   xvar <- tclVar("")
   if (two) yvar <- tclVar("")

   tt <- tktoplevel()
   tkwm.title(tt,"")
   x.entry <- tkentry(tt, textvariable=xvar)
   if (two) y.entry <- tkentry(tt, textvariable=yvar)

   reset <- function()
    {
     tclvalue(xvar)<-""
     if (two) tclvalue(yvar)<-""
    }

   reset.but <- tkbutton(tt, text="Reset", command=reset)

   submit <- function() {
     x <- as.character(tclvalue(xvar))
     if (two) y <- as.character(tclvalue(yvar))
     e <- parent.env(environment())
     e$x <- x
     if (two) e$y <- y
     tkdestroy(tt)
   }
   submit.but <- tkbutton(tt, text="Submit", command=submit)

   tkgrid(tklabel(tt,text=instructions),columnspan=2)
   tkgrid(tklabel(tt,text=box1), x.entry, pady = 10, padx =10)
   if (two) tkgrid(tklabel(tt,text=box2), y.entry, pady = 10, padx =10)
   tkgrid(submit.but, reset.but)

  tkwait.window(tt)
  if (two) {return (c(x,y))
  } else return (x)    
}

In [ ]:
# Ideas:
    1. Short-term CR more likely to be mechanism; Long-term CR more likely to be effects:
    
        a) get common signature for short-term CR/fasting --> CR mechanism
        b) get common signature for long-term CR --> CR mechanism + CR effects
        c) a & b (intersection) --> maintained CR mechanism
        d) a - b (difference) --> lost CR mechanism
        
    2. Find perturbagen combinations (>2) with large cumulative overlap:
    
        -define score for overlap between short-term CR signature and each perturbagen
        (may want to restrict search to GRAS/neutraceutical perturbagens)
        -choose perturbagen with largest overlap
        -re-define overlap for remaining short-term CR signature and each perturbagen
        -add perturbagen with largest overlap
        -repeat until threshold total overlap reached

In [2]:
#load libraries

library(GEOquery)
library(affy)
library(oligo)
library(stringr)
library(affxparser)
library(tcltk)
library(sva)
library(scatterplot3d)
library(limma)

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, grep, grepl,
    intersect, is.unsorted, lapply, lengths, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unlist, unsplit

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To c

In [3]:
#get raw data
gse_name <- "GSE60596"
data_dir <- paste(getwd(), "data", sep="/")


get_raw <- function (gse_name, data_dir) {
    #IN
    #OUT
    gse_dir <- paste(data_dir, gse_name, sep="/")
    #get raw data
    if (!file.exists(gse_dir)) {
        getGEOSuppFiles(gse_name, baseDir=data_dir)
    }
    #untar
    tar_name <- list.files(gse_dir, pattern="tar")
    untar(paste(gse_dir, tar_name, sep="/"), exdir=gse_dir)
    
    #unzip
    cel_paths <- list.files(gse_dir, pattern=".CEL.gz", full.names=T)
    sapply(cel_paths, gunzip, overwrite=T)   
}

In [4]:
get_raw(gse_name, data_dir)

/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483027_Mouse_EF_CD-1.CEL.gz 
                                                                                            11118245 
  /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483028_Mouse_EF_CD-2.CEL.gz 
                                                                                            11124001 
  /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483029_Mouse_EF_CD-3.CEL.gz 
                                                                                            11128249 
/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483030_Mouse_EF_CR55-1.CEL.gz 
                                                                                            11119305 
/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483031_Mouse_EF_CR55-2.CEL.gz 
                                                                                            11117817 
/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483032_Mouse_EF_CR55-3.CEL.gz 
                                                                                            11134701 
/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483033_Mouse_EF_CR70-1.CEL.gz 
                                                                                            11123421 
/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483034_Mouse_EF_CR70-2.CEL.gz 
                                                                                            11129797 
/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483035_Mouse_EF_CR70-3.CEL.gz 
                                                                                            11120861 
/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483036_Mouse_EF_CR85-1.CEL.gz 
                                                                                            11121325 
/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483037_Mouse_EF_CR85-2.CEL.gz 
                                                                                            11124189 
/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483038_Mouse_EF_CR85-3.CEL.gz 
                                                                                            11133609

In [5]:
load_eset <- function (gse_name, data_dir) {
    #IN
    #OUT
    gse_dir <- paste(data_dir, gse_name, sep="/")
    
    #get GSEMatrix (for pheno data)
    eset <- getGEO(gse_name, destdir=gse_dir, GSEMatrix=T)[[1]]
    
    #load celfiles and normalize
    cel_paths <- list.files(gse_dir, pattern=".CEL", full.names=T)
    data <- tryCatch (
        {
            raw_data <- ReadAffy (celfile.path=gse_dir)
            affy::rma(raw_data)
        },
        warning = function(cond) {
            raw_data <- read.celfiles(cel_paths)
            return (oligo::rma(raw_data))  
        } 
    )
    #rename samples in data
    sampleNames(data) <- str_extract(sampleNames(data), "GSM[0-9]+")
    
    #transfer exprs from data to eset (maintaining eset sample/feature order)
    sample_order <- sampleNames(eset)
    feature_order <- featureNames(eset)
    exprs(eset) <- exprs(data)[feature_order, sample_order]

    #add scan dates to pheno data (maintaining eset sample order)
    scan_dates <- cel_dates (cel_paths)
    names(scan_dates) <- sampleNames(data)
    pData(eset)$scan_date <- scan_dates[sample_order]
    
    return (eset)
}

In [6]:
eset <- load_eset(gse_name, data_dir)

ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE60nnn/GSE60596/matrix/
Found 1 file(s)
GSE60596_series_matrix.txt.gz
Using locally cached version: /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSE60596_series_matrix.txt.gz
Using locally cached version of GPL6246 found here:
/home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GPL6246.soft 
Loading required package: pd.mogene.1.0.st.v1
Loading required package: RSQLite
Loading required package: DBI
Platform design info loaded.


Reading in : /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483027_Mouse_EF_CD-1.CEL
Reading in : /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483028_Mouse_EF_CD-2.CEL
Reading in : /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483029_Mouse_EF_CD-3.CEL
Reading in : /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483030_Mouse_EF_CR55-1.CEL
Reading in : /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483031_Mouse_EF_CR55-2.CEL
Reading in : /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483032_Mouse_EF_CR55-3.CEL
Reading in : /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483033_Mouse_EF_CR70-1.CEL
Reading in : /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483034_Mouse_EF_CR70-2.CEL
Reading in : /home/alex/Documents/Batcave/GEO/caloric-restriction/data/GSE60596/GSM1483035_Mouse_EF_CR70-3.CEL
Reading

In [11]:
adjust_batch <- function(eset) {
    #IN
    #OUT
    choices <- paste(sampleNames(eset), pData(eset)$title)
    level_names <- c()

    #repeat until all level (except control) selected for each variable
    while (TRUE) {
        #select level
        group <- tk_select.list(choices, multiple=T, title="Select samples in each level (except control) of variable")
        group <- str_extract(group, "GSM[0-9]+")
        if (length(group) == 0) break

        #add level to pheno
        level_name <- inputs(box1="Level Name")
        pData(eset)[, level_name] <- 0
        pData(eset)[group, level_name] <- 1

        #add to var_names
        level_names <- c(level_names, level_name)

    }
    #generate model matrix
    levels <- paste(level_names, collapse="+")
    formula <- as.formula(paste("~", levels, sep=""))
    model <- model.matrix(formula, data=pData(eset))
        
    #correct for scan date effects (ComBat)
    message("Correcting for scan date (ComBat)")
    batch <- pData(eset)$scan_date
    exprs <- exprs(eset)
    combat_exprs <- ComBat(exprs, batch, model)
    exprs(eset) <- combat_exprs
    return (eset) 
}

In [12]:
adj_eset <- adjust_batch(eset)

Correcting for scan date (ComBat)


Found 3 batches
Adjusting for 3 covariate(s) or covariate level(s)
Standardizing Data across genes
Fitting L/S model and finding priors
Finding parametric adjustments
Adjusting the Data


In [18]:
# sampleNames <- c("AL liver 1", "AL liver 2", "CR liver 1", "CR liver 2",
#                  "AL heart 1", "AL heart 2", "CR heart 1", "CR heart 2")

# titles <- c("GSM1", "GSM2", "GSM3", "GSM4",
#             "GSM5", "GSM6", "GSM7", "GSM8")

# pheno <- as.data.frame(sampleNames, titles)

# choices <- paste(sampleNames, titles)
# contrasts <- c()
# group_levels <- c()
# selected_samples <- c()

# #repeat above until no AL samples selected
# while (TRUE) {
    
#     #select AL samples
#     AL <- tk_select.list(choices, multiple=T, title="select AL (control) samples")
#     AL <- str_extract(AL, "GSM[0-9]+")
#     if (length(AL) == 0) break

#     #select CR samples
#     CR <- tk_select.list(choices, multiple=T, title="select CR (test) samples")
#     CR <- str_extract(CR, "GSM[0-9]+")
        
#     #add group names to pheno
#     AL_name <- inputs("Enter AL group name")
#     CR_name <- inputs("Enter CR group name")
#     pheno[AL, "group"] <- AL_name
#     pheno[CR, "group"] <- CR_name   
        
#     #add to contrasts
#     contrasts <- c(contrasts, paste(CR_name, AL_name, sep="-"))

#     #add to group_levels
#     group_levels <- unique(c(group_levels, AL_name, CR_name))

#     #add to selected_samples
#     selected_samples <- unique(c(selected_samples, AL, CR))
# }
# #retain selected samples only
# eset <- eset[, selected_samples]

# #generate model and contrast matrix
# group <- factor(pheno$group, levels=group_levels)
# model <- model.matrix(~0+group)
# colnames(model) <- group_levels
# contrast_matrix <- makeContrasts(contrasts=contrasts, levels=model)

In [15]:
diff_expr <- function (eset) {
    #INPUT:
    #OUTPUT:
    
    choices <- paste(sampleNames(eset), pData(eset)$title)
    contrasts <- c()
    group_levels <- c()
    selected_samples <- c()

    #repeat until all AL samples selected
    while (TRUE) {
        #select AL samples
        AL <- tk_select.list(choices, multiple=T, title="select AL (control) samples")
        AL <- str_extract(AL, "GSM[0-9]+")
        if (length(AL) == 0) break

        #select CR samples
        CR <- tk_select.list(choices, multiple=T, title="select CR (test) samples")
        CR <- str_extract(CR, "GSM[0-9]+")

        #add group names to pheno
        group_names <- inputs("Enter group names", two=T)
        pData(eset)[AL, "group"] <- group_names[1]
        pData(eset)[CR, "group"] <- group_names[2]

        #add to contrasts
        contrasts <- c(contrasts, paste(group_names[2], group_names[1], sep="-"))

        #add to group_levels
        group_levels <- unique(c(group_levels, group_names[1], group_names[2]))
            
        #add to selected_samples
        selected_samples <- unique(c(selected_samples, AL, CR))
    }
    #retain selected samples only
    eset <- eset[, selected_samples]
        
    #generate model and contrast matrix
    group <- factor(pData(eset)$group, levels=group_levels)
    model <- model.matrix(~0+group)
    print (model)
    colnames(model) <- group_levels
    contrast_matrix <- makeContrasts(contrasts=contrasts, levels=model)
    print (contrast_matrix)

    
#     #PCA for CR/AL
#     colors <- ifelse(group == "AL", "red", "blue")
#     pca <- prcomp(t(exprs(eset)))

#     s3d <- scatterplot3d(pca$x[,1], pca$x[,2], pca$x[,3], 
#                          main = gse_name, 
#                          color = colors,
#                          pch=19, type="h", lty.hplot=3)

#     s3d.coords <- s3d$xyz.convert(pca$x[,1], pca$x[,2], pca$x[,3])
#     legend("topleft", inset=.05, 
#            bty="n", cex=.8,
#            title="Diet",
#            c("AL", "CR"), fill=c("red", "blue"))
     
    #differential expression (limma)
    fit <- contrasts.fit (lmFit(exprs(eset), model), contrast_matrix)
    ebayes <- eBayes(fit)
    top_genes <- topTable(ebayes, coef=1, n=Inf, resort.by="logFC", p.value=0.05)
    
    return (top_genes)
}


In [16]:
top_genes <- diff_expr(adj_eset)

  groupAL groupCR
1       1       0
2       1       0
3       1       0
4       0       1
5       0       1
6       0       1
7       0       1
8       0       1
9       0       1
attr(,"assign")
[1] 1 1
attr(,"contrasts")
attr(,"contrasts")$group
[1] "contr.treatment"

      Contrasts
Levels CR-AL
    AL    -1
    CR     1
